# Het Viterbi algoritme toegepast op participant wisselingen in de Psalmen

### Bronnen die ik gebruikt heb om dit idee uit te werken: 

http://www.cs.columbia.edu/~mcollins/ - "Hidden Markov models and tagging (sequence labeling) problems"

http://www.nltk.org/_modules/nltk/parse/viterbi.html#ViterbiParser

http://stackoverflow.com/questions/9729968/python-implementation-of-viterbi-algorithm 

Source of code below: https://en.wikipedia.org/wiki/Viterbi_algorithm

### Inleiding
Gegeven is de volgende voorbeeld code, geschreven om de kans op een bepaalde reeks van 'hidden states' (gezond of ongezond) te achterhalen. De observaties zijn 'normaal', 'koud', of 'duizelig'. In onderstaande kernels geef ik aan hoe ik het Viterbi algoritme toegepast zou zien op participant wisselingen (PNG) in de Psalmen. Omdat er voorbereidende (kans)berekeningen gedaan dienen te worden, met bijbehorende code, beschrijf ik aan het einde van deze notebook een aantal (methodische) vragen. 

### Toepassing

In [2]:
states = ('Healthy', 'Fever') 
'''Verborgen maar expliciete subjecten, objecten of complementen, dat is een voorgegeven set in de Psalmen.''' 

observations = ('normal', 'cold', 'dizzy') 
'''De observaties in een clause/sentence zijn in het geval van de Psalmen:
['1Csg']
['1Csg']
['1Cpl']
['2Msg']
['2Fsg']
['2Mpl']
['2Fpl']
['3Msg']
['3Msg']
['3Fsg']
['3Mpl']
['3Mpl']
['3Mpl']           
['3Fpl']
['3Fpl']
['absent']
['n/a']
Om het model eenvoudiger te maken, kan in eerste instantie uitgegaan worden van bijvoorbeeld 1e, 2e en 3e persoon. 
Daarna kan eventueel getest worden met ingewikkelder 'observaties', i.e. 1Csg of 3Mpl. 
Hier kunnen (al dan niet geautomatiseerd) reeksen van PNG wisselingen uit een clause/sentence/tekst ingevoerd worden, 
om zo de kans op bepaalde verborgen participanten ('states') te bepalen.
'''

start_probability = {'Healthy': 0.6, 'Fever': 0.4}
'''Dit is de start kans op een bepaald expliciet subject of object in een clause/sentence.'''

transition_probability = {
'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
'Fever' : {'Healthy': 0.4, 'Fever': 0.6}
    }
'''Dit is de overgangskans van een bepaald subject naar een ander subject in een clause/sentence. 
Dit geldt ook voor objecten.'''

emission_probability = {
'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}
    }
'''Dit geeft de kans aan op een bepaalde persoonvorm bij een bepaald subject. 
Bijvoorbeeld: als de verborgen staat "YHWH" is dan is de kans op 1e persoon 0.6.'''

In [5]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for i in states:
        V[0][i] = start_p[i]*emit_p[i][obs[0]]
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for y in states:
            prob = max(V[t - 1][y0]*trans_p[y0][y]*emit_p[y][obs[t]] for y0 in states)
            V[t][y] = prob
    for i in dptable(V):
        print(i)
    opt = []
    for j in V:
        for x, y in j.items():
            if j[x] == max(j.values()):
                opt.append(x)
    # The highest probability
    h = max(V[-1].values())
    print('The steps of states are ' + ' '.join(opt) + ' with highest probability of %s'%h)

def dptable(V):
    # Print a table of steps from dictionary
    yield " ".join(("%10d" % i) for i in range(len(V)))
    for y in V[0]:
        yield "%.7s: " % y+" ".join("%.7s" % ("%f" % v[y]) for v in V)

In [6]:
viterbi(observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability)

         0          1          2
Healthy: 0.30000 0.08400 0.00588
Fever: 0.04000 0.02700 0.01512
The steps of states are Healthy Healthy Fever with highest probability of 0.01512


### Een beetje theorie

De analyse van de PNG wisselingen bevindt zich op het snijvlak van syntax en semantiek. Geen van de bestaande grammatica's besteed serieus aandacht aan PNG wisselingen omdat de identificatie van patronen en uiteindelijk participanten het clause niveau ontstijgt. De meeste grammatica's stoppen bij het clause niveau. De analyse van enkele persoon, geslacht, getal kenmerken op clause niveau is snel gedaan, de identificatie vindt plaats op sentence, verse niveau of zelfs pas in alinea's, of teksten in het geheel. Bij die laatse stap, de identificatie, komt 'kennis van de wereld', van de tekst-context kijken. 

Dat laatste is belangrijk omdat iemand die een tekst probeert te begrijpen de meest waarschijnlijke analyse van participanten probeert waar te nemen, op basis van de frequenties die hij/zij eerder heeft waargenomen in de analyse van eerdere participantwisselingen in andere teksten (vrij naar: Rens Bod, Beyond Grammar, 1998). De lezer maakt een analyse op basis van een 'kansberekening'. De lezer maakt een inschatting van wat de beste identificatie is. 

Nu wil ik met mijn onderzoek het verstaansproces van een lezer niet nabootsen. De gedachte echter dat een lezer een kansinschatting maakt van wat mogelijkerwijs de beste identificatie van een participant is, op basis van eerder tegengekomen reeksen van participanten (lees: patronen van PNG wisselingen) is waardevol. 

Wil een lezer een goede kansinschatting maken dan heeft hij de juiste informatie nodig. In de Psalmen is veel informatie reeds gegeven. Er zijn explicitiete subjecten en objecten (zie onder), er zijn een N, Q, en D domeinen, er is taalkundige data zoals persoon, geslacht, getal kenmerken.

Met name de combinatie van informatie over expliciete subjecten (en expliciete objecten, voor zover je daar over kunt spreken) kan toegepast worden in het Viterbi algoritme.

Met R > summary(data2$subj). Onderstaande samenvatting geeft de meest voorkomende subjecten weer voor de Psalmen. 

          _        JHWH         >TH         >NJ       >LHJM        NPCJ          MJ 
       3589          93          58          36          30          24          22 
        HW>         LBJ         HMH       RC<JM       H_>RY        <JNJ       JFR>L 
         20          19          15          14          12          10          10 
      JMJNK          KL          >C        >DNJ        GWJM        HRJM         MJM 
          9           9           8           8           8           8           8 
         PJ         >DM        >WJB         JDK        <MJM         >RY        YDJQ 
          8           7           7           7           6           6           6 
       ZDJM        <YMJ          >L       >WJBJ        RWXJ        XSDK      YDJQJM 
          6           5           5           5           5           5           5 
      <NWJM        >JBJ         >JC       >NXNW        CMJM        FPTJ          JH 
          4           4           4           4           4           4           4 
    KL_GWJM KL_P<LJ_>WN         KXJ       LCWNJ       NHRWT       NPCNW         RC< 
          4           4           4           4           4           4           4 
       RGLJ       TPLTJ        <BDK       <JNJW        BFRJ        BNJW         CMW 
          4           4           3           3           3           3           3 
      H_MLK        J<QB       JCRJM KL_>PSJ_>RY         KLM       LBBKM        LJLH 
          3           3           3           3           3           3           3 
       NPCM        NPCW        XSDW         YRJ         Z>T        ZR<W         <CN 
          3           3           3           3           3           3           2 
      <FJHM       <JNJK         <MJ    <MJM_KLM       <WNTJ     >BWTJNW       >JBJK 
          2           2           2           2           2           2           2 
   >K_>LHJM    >KLJ_<MJ       >MRTK        >NKJ        >NWC     >P_JHWH      >WJBJK 
          2           2           2           2           2           2           2 
    BNJ_NKR        BNJK        BNJM  BNWT_JHWDH        DBRW        FRJM       GXLJM 
          2           2           2           2           2           2           2 
      H_CMC      H_CMJM      H_GWJM        H_JM H_JM_W_ML>W         JDJ      JDJDJK 
          2           2           2           2           2           2           2 
      JR>JK     (Other) 
          2         581 
          

Met R > summary(data2$obj). Onderstaande samenvatting geeft de meest voorkomende objecten weer voor de Psalmen. 

                        _                      NPCJ                        JH 
                     3212                        39                        30 
                  >T_JHWH                      JHWH                      PNJK 
                       24                        18                        18 
                      CMK                       >RY                     TWRTK 
                       17                        13                        13 
                     XQJK                     TPLTJ                       VWB 
                       12                        10                        10 
                     <MJM                      >ZNK                        MH 
                        8                         8                         8 
                   PQWDJK                      XSDK                       <MW 
                        8                         8                         7 
                    >LHJM                       YDQ                     <DTJK 
                        7                         7                         6 
                       <Z                     >MRTK                   CJR_XDC 
                        6                         6                         6 
                    H_>RY                        KL                       LBJ 
                        6                         6                         6 
                   MYWTJK                      PNJW                      QWLJ 
                        6                         6                         6 
                       R<                       RC<                     YDQTK 
                        6                         6                         6 
                      Z>T                       <MK                        >L 
                        6                         5                         5 
               >T_CM_JHWH                     BRJTW                       CMW 
                        5                         5                         5 
                      CW>                      DRKJ                      DRKK 
                        5                         5                         5 
                     GWJM                       KPJ                      MCPV 
                        5                         5                         5 
                     NDRJ                      RGLJ                       TBL 
                        5                         5                         5 
                      XJL                        ZH                      <BDK 
                        5                         5                         4 
                     <JNJ                       <ZK                       >LH 
                        4                         4                         4 
                      >WN                      CLWM                     LCWNM 
                        4                         4                         4 
                   MCPVJK                       MJM                     PQDJK 
                        4                         4                         4 
               QWL_TXNWNJ                       R<H                       R>C 
                        4                         4                         4 
XSDW_W_NPL>WTJW_L_BNJ_>DM                      YJWN                       <JR 
                        4                         4                         3 
                      <NJ                     <NWJM                       <WN 
                        3                         3                         3 
                    >BJWN                       >TW                      >WTM 
                        3                         3                         3 
                      >XT                       CKM                      CMJM 
                        3                         3                         3 
                      CQR                       DBR                      DBRK 
                        3                         3                         3 
                     DBRW                       JDJ                       JDK 
                        3                         3                         3 
                       JM                  KBWD_CMW                      KS>W 
                        3                         3                         3 
                     LBBJ                       LBM                       LXM 
                        3                         3                         3 
                   MJCRJM                     MLKJM                      NPCM 
                        3                         3                         3 
                     NPCW                        PJ                      PJHM 
                        3                         3                         3 
                       PX                       QRN                     THLTK 
                        3                         3                         3 
                      XCK                      XRPH                      YDJQ 
                        3                         3                         3 
                  (Other) 
                     1049 

Met R > summary(data2$cmpl). Onderstaande samenvatting geeft de meest voorkomende complementen weer voor de Psalmen. 

           _           LJ       L_JHWH          <LJ           LK         >LJK           LW 
        3256           65           51           45           41           31           30 
        MMNJ           BW           BK          LHM      >L_JHWH           BJ       B_JHWH 
          24           22           21           20           18           18           16 
         LNW          >LJ          LMW      L_>LHJM       L_PNJK       L_PNJW        <LJHM 
          16           14           14           12           10            9            8 
        MMNW         <LJK        <LJNW         <LJW         >LJW      B__GWJM      B_>LHJM 
           8            7            7            7            7            7            7 
         BHM       M_CMJM       L__>RY       L_C>WL        B_CMK        B_LBW           BM 
           7            7            6            6            5            5            5 
       L_<MW      L_<ZRTJ        L_CMK        L_DWD          MMK          <MW           BH 
           5            5            5            5            5            4            4 
    L_CKNJNW       L_NPCJ           LH        M_>RY        M_MWT      <L_JHWH        <LJMW 
           4            4            4            4            4            3            3 
     B__<MJM        B__>C       B__>RY      B__MDBR       B_>HLK       B_>MTK     B_JCW<TK 
           3            3            3            3            3            3            3 
       B_LBJ        B_LBM       B_XQJK      B_YDQTK   B_YL_KNPJK       L__<PR       L_<BDK 
           3            3            3            3            3            3            3 
      L_DBRK        L_MCH   L_PNJ_JHWH       M_<WNJ         M_R<      M_TWRTK          MNJ 
           3            3            3            3            3            3            3 
     <D_>DWM      <L_<BDK      <L_>DWM      <L_CMJM     <L_H_>RY    <L_H_CMJM     <L_JR>JW 
           2            2            2            2            2            2            2 
    <L_NPCNW         <LJH     >L_>LHJM >L_HJKL_QDCK      >T_PNJK        >XRJK         >XWR 
           2            2            2            2            2            2            2 
      B__>DM      B__CMJM      B__HRJM      B_>MRTK   B_BJT_JHWH    B_CM_QDCW        B_DRK 
           2            2            2            2            2            2            2 
    B_JCW<TW        B_JDK      B_JFR>L     B_JM_SWP    B_KL_P<LK  B_M<FJ_JDJK     B_MYWTJK 
           2            2            2            2            2            2            2 
    B_NDJBJM      (Other) 
           2          840 

### Vragen 

Ik heb jammer genoeg niet zo veel verstand van statistiek, en voor de toepassing van het Viterbi algoritme moeten er eerst drie zaken berekend worden: 

1. start_probability
2. transition_probability
3. emission_probability

Ik heb daarom een aantal vragen: 

1. Is het mogelijk deze kansen te berekenen op basis van expliciete object/subject informatie in de Psalmen, of misschien zelfs de hele HB? 
2. Zo ja, hoe?
3. Hoe kan ik dit Viterbi-participanten model uitwerken en toepassen? 
4. Heb jij, Dirk, interesse om met mij hier aan te werken, wellicht voor een samenwerkings paper? 